In [ ]:
import cv2
import os
from pathlib import Path
import random as rd
import numpy as np
from PIL import Image

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import applications
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

from hulp_functies import plot_model, plot_images, generate_metadata
rd.seed(42)

### Resize de foto's
Zorg dat de gemaakte foto's de grootte van het neurale netwerk hebben (224 x 224 x 3). En splits de foto's van de voorwerpen in een training set en een test set.

De code in de volgende twee cellen leest de plaatjes uit de folder `camera`, resizet ze naar het juiste formaat en splitst ze op in een folder `train` en een folder `test`. Het resultaat komt in de folder `images_224`.

De plaatjes uit `train` worden gebruikt om het neurale netwerk te trainen, de plaatjes uit `test` worden gebruikt om te kijken hoe goed de training gelukt is.

In [ ]:
size = 224
channels = 3
input_shape = (size, size, channels)

camera_dir = Path('camera')
voorwerp_dir = camera_dir / 'voorwerpen'
voorwerp_fn = Path('voorwerpen.txt')
achtergrond_fn = 'achtergrond'
resize_dir = Path(f'images_{size}')

classes = os.listdir(voorwerp_dir)
n_classes = len(classes)
class_nums = {c:i for i,c in enumerate(classes)}

with open(voorwerp_fn, 'w') as f:
    f.write('\n'.join(classes))    

In [ ]:
(resize_dir / achtergrond_fn).mkdir(exist_ok=True, parents=True)
for c in classes:
    (resize_dir / 'train' / c).mkdir(exist_ok=True, parents=True)
    (resize_dir / 'test'  / c).mkdir(exist_ok=True, parents=True) 

for c in classes:
    ims = os.listdir(voorwerp_dir / c)
    rd.shuffle(ims)
    Ntest = len(ims) // 5
    for i, im in enumerate(ims):
        dir_name = 'test' if i < Ntest else 'train'
        image = cv2.imread(str(voorwerp_dir / c / im))
        image_resized = cv2.resize(image, (size, size))
        cv2.imwrite(str(resize_dir / dir_name / c / f'{str(i)}.png'), image_resized)
            
for i, im in enumerate(os.listdir(camera_dir / achtergrond_fn)):
    image = cv2.imread(str(camera_dir / achtergrond_fn / im))
    image_resized = cv2.resize(image, (size, size))
    cv2.imwrite(str(resize_dir / achtergrond_fn / f'{str(i)}.png'), image_resized)    

### Train het neurale netwerk

#### Maak de folders
Tijdens het trainen make we een aantal modellen en een aantal grafieken. Deze modellen en grafieken worden in aparte folders gezet. De code in de volgende cel maakt die folders.

In [ ]:
keras_name = 'model_mobilenet.h5'
lite_name = 'model_mobilenet.tflite'
json_name = 'model_mobilenet.json'

model_dir = Path('models')
saved_model_dir = Path('saved_models')
export_model_dir = Path('export_models')
plot_dir = Path('plots')

model_dir.mkdir(exist_ok=True)
saved_model_dir.mkdir(exist_ok=True)
export_model_dir.mkdir(exist_ok=True)
plot_dir.mkdir(exist_ok=True)

#### Lees de data

De foto's die je geplaatst hebt in de folder `camera` zijn ge-resized en in de folder `images_224` gezet. De volgende twee cellen lezen deze plaatjes in en zetten ce in een array in het werkgeheugen van je computer.

In [ ]:
def import_voorwerpen(p):
    images = {}
    
    for c in classes:
        class_dir = p / c
        im_names = os.listdir(class_dir)
        images[c] = np.zeros((len(im_names), size, size, channels))
        for i, img in enumerate(im_names):
            im = np.asarray(Image.open(class_dir / img))
            images[c][i] = im/255

    data = np.concatenate([images[c] for c in classes], axis=0)
    labels = []
    for c in classes:
        labels += [class_nums[c]]*len(images[c])
    labels = np.array(labels)
    
    return data, labels, to_categorical(labels, n_classes)

def import_achtergrond(p, N=60):
    im_names = os.listdir(p)
    NN = min(N, len(im_names))

    npd_images = np.zeros((NN, size, size, channels))
    for i, img in enumerate(im_names[:NN]):
        im = np.asarray(Image.open(p / img))
        npd_images[i] = im/255
    class_npd = np.full((npd_images.shape[0], n_classes), 1/n_classes)

    return npd_images, class_npd

In [ ]:
training_data, training_labels, class_train = import_voorwerpen(resize_dir / 'train')
test_data, test_labels, class_test = import_voorwerpen(resize_dir / 'test')

Ntrain = training_data.shape[0]
Ntest = test_data.shape[0]

npd_train_images, class_npd_train = import_achtergrond(resize_dir / achtergrond_fn, N=2*Ntrain)

training_data_ext = np.concatenate((training_data, npd_train_images), axis=0)
class_train_ext = np.concatenate((class_train, class_npd_train), axis=0)

#### Maak een model
We nemen een standaard model: Mobilenet. Dit model is apart ontworpen voor mobile devices zoals smartphones. Het enige wat we dit model nog moeten vertellen is hoeveel soorten objecten het moet kunnen onderscheiden.

In [ ]:
base_model = applications.MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)

x = base_model.output
x = Flatten()(x)

predictions = Dense(n_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

#### Een functie om het model te trainen

In [ ]:
def train_model(m, tr_data, tr_class, bs, epochs, lr=1.0e-4, fn=None):
    cp = ModelCheckpoint(str(saved_model_dir / fn),
                         monitor='val_loss',
                         verbose=0, # verbosity - 0 or 1
                         save_best_only= True,
                         mode='auto')

    m.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr),
              metrics = ['accuracy'])
    
    details = m.fit(tr_data, tr_class,
                    batch_size = bs,
                    epochs = epochs,
                    shuffle = True,
                    validation_data= (test_data, class_test),
                    callbacks=[cp],
                    verbose=1)
    if not fn is None:
        m.save(str(model_dir / fn))
    return details

#### Train het model
Nu komt het echte werk: de training van het model. De batch size is 32, dat betekent dat het model telkens 32 plaatjes bekijkt en daarop zijn gewichten aanpast. Het aantal epochs is 6, dat betekent dat alle plaatjes 6 keer worden bekeken.

Deze stap kan lang duren, afhankelijk van de rekenkracht van je computer.

In [ ]:
%%time

batch_size = 32
epochs = 6

model_details = train_model(model, training_data_ext, class_train_ext, bs=batch_size, epochs=epochs, fn=keras_name)

#### Bekijk de training
Deze grafieken laten zien hoe goed het model was na iedere epoch (een epoch was 1 keer alle trainings plaatjes bekijken).

In [ ]:
plot_model(model_details, plot_dir / "model_details.png")

#### Bekijk het resultaat
We kunnen kijken hoe goed het model is op de test plaatjes.

In [ ]:
class_pred = model.predict(test_data)
labels_pred = np.argmax(class_pred,axis=1)
print(f'nauwkeurigheid op de test plaatjes: {100*np.mean(labels_pred==test_labels):.2f}%')

idx = rd.sample(range(Ntest), 12)
plot_images(test_data[idx], test_labels[idx], classes, labels_pred[idx])

#### Converteer het model naar een tensorflow lite model
Om het model in een Android app te kunnen gebruiken moet het naar een iets ander formaat worden omgezet. De gewichten in het model krijgen iets minder bits waardoor het model iets kleiner wordt.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

(model_dir / lite_name).write_bytes(tflite_model)

#### Voeg metadata toe
De Android app moet nog een paar dingen weten ocer het models, zoals wat het input formaat is en hoeveel soorten objecten het moet kunnen onderscheiden. Deze informatie komt in een aparte json file. Je kunt ook je eigen naam hier in zetten als auteur.

In [ ]:
MODEL_INFO = {
    'author' : '<je eigen naam>',
    'size' : size,
    'classes' : classes,
    'label_fn' : str(voorwerp_fn),
    'label_path' : voorwerp_fn.resolve(),
    'model_path' : (model_dir / lite_name).resolve(),
    'export_model_path' : (export_model_dir / lite_name).resolve(),
    'json_fn' : export_model_dir / json_name
}
generate_metadata(MODEL_INFO)